In [5]:
from CNNectome.validation.organelles.segmentation_metrics import Evaluator,EvaluationMetrics,display_name,sorting
import h5py
import numpy as np
import csv
import os
import matplotlib
%matplotlib inline 
from matplotlib import pyplot as plt
import pandas as pd
import re
import zarr
import tifffile
import seaborn
import pandas
from scipy.ndimage import label, distance_transform_edt
from CNNectome.utils.compute_label_distribution import distance
import scipy

/groups/scicompsoft/home/ackermand/miniconda3/envs/annotator-metrics/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/groups/scicompsoft/home/ackermand/miniconda3/envs/annotator-metrics/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/groups/scicompsoft/home/ackermand/miniconda3/envs/annotator-metrics/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)ty